In [1]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/cpu:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 10605655448826256541, name: "/gpu:0"
 device_type: "GPU"
 memory_limit: 11332668621
 locality {
   bus_id: 2
 }
 incarnation: 16019460948262561335
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:85:00.0"]

In [5]:
import os, glob, bcolz

import numpy as np
import pandas as pd

from tqdm import tqdm
from scipy import ndimage, misc

from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import roc_auc_score

from keras import backend as K
from keras import optimizers
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator

from keras.layers.normalization import BatchNormalization
from keras.layers import Dense, Dropout, Flatten, Activation, Input
from keras.layers.convolutional import MaxPooling2D, Convolution2D
from keras.layers.advanced_activations import PReLU, LeakyReLU

from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.applications.inception_v3 import preprocess_input as preprocess_input_incep_xcep
from keras.applications.imagenet_utils import preprocess_input as preprocess_input_vgg_resnet

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

In [35]:
def read_img(img_path, img_shape):
    img = misc.imread(img_path)
    img = misc.imresize(img, img_shape)
    return img

def read_imgs(img_height, img_width):
    train_img, test_img = [],[]
    for img_path in tqdm(train_set['name'].iloc[:]):
        train_img.append(read_img(os.path.join(path, 'train', str(img_path)+'.jpg'), (img_height, img_width)))

    for img_path in tqdm(test_set['name'].iloc[:]):
        test_img.append(read_img(os.path.join(path, 'test', str(img_path)+'.jpg'), (img_height, img_width)))
    return np.array(train_img), np.array(test_img)

def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()

def load_array(fname):
    return bcolz.open(fname)[:]

def freeze_model(model):
    for layer in model.layers:
        layer.trainable = False
    return model

def grab_optimizer(opt, lr):
    if opt == 'sgd':
        return optimizers.SGD(lr=lr, decay=1e-6, momentum=0.8, nesterov=True)
    elif opt == 'adam':
        return optimizers.Adam(lr=lr)
    elif opt == 'adagrad':
        return optimizers.Adagrad(lr=lr)
    elif opt == 'rmsprop':
        return optimizers.RMSprop(lr=lr)
    
def dense_block(units, activation, drop_prob, inputs):
    x = Dense(units, activation=None)(inputs)
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    x = Dropout(drop_prob)(x)
    return x

def make_conv_model(input_shape, optimizer):
    inputs = Input(shape=input_shape)
    m = conv_block(16, (3,3), (2,2),'relu', inputs=inputs)
    m = conv_block(32, (3,3), (2,2), 'relu', inputs=m)
    m = conv_block(64, (3,3), (2,2), 'relu', inputs=m)
    m = conv_block(128, (3,3), (2,2), 'relu', inputs=m)
    m = conv_block(256, (3,3), (2,2), 'relu', inputs=m)
    m = Flatten()(m)
    m = dense_block(2048, 'relu', 0.25, inputs=m)
    m = dense_block(512, 'relu', 0.5, inputs=m)
    outputs = dense_block(1, 'sigmoid', 0, inputs=m)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

def make_vgg19_conv(input_shape):
    base_model = VGG19(input_shape=input_shape, weights='imagenet', include_top=False)
    base_model = freeze_model(base_model)
    return base_model

def make_incepv3_conv(input_shape):
    base_model = InceptionV3(input_shape=input_shape, weights='imagenet', include_top=False)
    base_model = freeze_model(base_model)
    return base_model

def make_resnet50_conv(input_shape):
    base_model = ResNet50(input_shape=input_shape, weights='imagenet', include_top=False)
    base_model = freeze_model(base_model)
    return base_model

def make_xception_conv(input_shape):
    base_model = Xception(input_shape=input_shape, weights='imagenet', include_top=False)
    base_model = freeze_model(base_model)
    return base_model

def make_ft_dense(input_shape, optimizer):
    inputs = Input(shape=input_shape)
    m = Flatten()(inputs)
    m = dense_block(1024, 'relu', 0.25, inputs=m)
    m = dense_block(1024, 'relu', 0.5, inputs=m)
    outputs = dense_block(1, 'sigmoid', 0, inputs=m)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

def make_ft_rand_forest(random_state=100, n_estimators=1000):
    return RandomForestClassifier(random_state=random_state, n_jobs=-1, n_estimators=n_estimators)

def make_ft_svm():
    pass

def train_skmodel_ft(model, base_model, train_data, train_label, model_out, model_init_weights, kfolds):
    kf = KFold(n_splits=kfolds, shuffle=True)
    
    i = 0
    models_stats = {}
    for train_ixs, valid_ixs in kf.split(train_data):
        # use convolutional model to precompute training
        x_train = train_data[train_ixs]
        x_valid = train_data[valid_ixs]
        y_train = train_label[train_ixs]
        y_valid = train_label[valid_ixs]

        x_train = base_model.predict(x_train)
        x_valid = base_model.predict(x_valid)

        x_train = x_train.reshape((x_train.shape[0], np.prod(x_train.shape[1:])))
        x_valid = x_valid.reshape((x_valid.shape[0], np.prod(x_valid.shape[1:])))
                
        #re-initialzie the weights of the model on each run
        model = joblib.load(model_init_weights)
        model_out_file = '{}_{}.pkl'.format(model_out, str(i))

        model.fit(x_train, y_train)
        joblib.dump(model, model_out_file)
        
        eval_tr = model.score(x_train, y_train)
        eval_va = model.score(x_valid, y_valid)
        
        tr_score = roc_auc_score(y_train, model.predict(x_train))
        va_score = roc_auc_score(y_valid, model.predict(x_valid))
        
        print('\n')
        print('kfold: {}'.format(str(i)))
        print('best model train acc: {}'.format(eval_tr))
        print('best model valid acc: {}'.format(eval_va))
        print('best model train aroc score: {}, valid aroc score: {}'.format(tr_score, va_score))
        print('\n')
        models_stats[model_out_file] = {'score_tr_va':[tr_score, va_score], 'train_acc':[eval_tr], 'val_acc':[eval_va]}
        
        with open(os.path.join(models_path,'{}_{}.out'.format(model_out,'history')), 'a') as f:
            f.write('kfold: {}'.format(str(i)))
            f.write('best model train acc: {}'.format(eval_tr))
            f.write('best model valid acc: {}'.format(eval_va))
            f.write('best model train aroc score: {}, valid aroc score: {}'.format(tr_score, va_score))
            f.write('\n')
        
        i += 1
    
    return models_stats

setup data dirs and read in imgs:

In [7]:
DATA_DIR = os.path.join('/scratch', 'yns207', 'data_invasive')

path = DATA_DIR
test_path = os.path.join(path, 'test')
models_path = os.path.join(path, 'results')
train_path = os.path.join(path, 'train')
valid_path = os.path.join(path, 'valid')
print('DATA_PATH: ' + path)

DATA_PATH: /scratch/yns207/data_invasive


# loading data

In [ ]:
train_set = pd.read_csv(os.path.join(path, 'train_labels.csv'))
test_set = pd.read_csv(os.path.join(path, 'sample_submission.csv'))

In [ ]:
train_img, test_img = read_imgs(300,400)
train_label = np.array(train_set['invasive'].iloc[:])

create a holdout set of 10%:

In [ ]:
train_img, hold_img, train_labels, hold_labels = train_test_split(train_img, train_label, test_size=0.10)

In [ ]:
train_img.shape, hold_img.shape, train_labels.shape, hold_labels.shape

save the datasets unaltered so they can be loaded again at a later point:

In [ ]:
%cd $models_path
save_array('aug_3_train_img.dat', train_img)
save_array('aug_3_hold_img.dat', hold_img)
save_array('aug_3_train_labels.dat', train_labels)
save_array('aug_3_hold_labels.dat', hold_labels)

# checkpoint

read the datasets with bcolz:

In [8]:
%cd $models_path
train_img = load_array('aug_3_train_img.dat')
hold_img = load_array('aug_3_hold_img.dat')
train_labels = load_array('aug_3_train_labels.dat')
hold_labels = load_array('aug_3_hold_labels.dat')

/scratch/yns207/data_invasive/results


In [9]:
train_img.shape, hold_img.shape, train_labels.shape, hold_labels.shape

((2065, 300, 400, 3), (230, 300, 400, 3), (2065,), (230,))

In [10]:
batch_size = 32
epochs = 100
kfolds = 5
lr = 0.00025

In [ ]:
# get it into right folder
%cd $models_path
model_name = 'invasive_vgg19_rf_aug4'
init_weights_model = '{}_base.pkl'.format(model_name)

# create base model
base_model = make_vgg19_conv(train_img[0].shape)

# train dense model ontop of precomputed conv features
rand_forest = make_ft_rand_forest(random_state=100, n_estimators=1000)
joblib.dump(rand_forest, init_weights_model)

#preprocess imgs
vgg19_train_img = preprocess_input_vgg_resnet(train_img.astype(np.float32))

# train dense model on folds
performance = train_skmodel_ft(rand_forest, base_model, vgg19_train_img, train_labels, model_name, init_weights_model, kfolds)

/scratch/yns207/data_invasive/results


kfold: 0
best model train acc: 1.0
best model valid acc: 0.9031476997578692
best model train aroc score: 1.0, valid aroc score: 0.8953105196451204




In [ ]:
performance

In [ ]:
# get it into right folder
%cd $models_path
model_name = 'invasive_inceptionv3_rf_aug4'
init_weights_model = '{}_base.pkl'.format(model_name)

# create base model
base_model = make_incepv3_conv(train_img[0].shape)

# train dense model ontop of precomputed conv features
rand_forest = make_ft_rand_forest(random_state=100, n_estimators=1000)
joblib.dump(rand_forest, init_weights_model)

#preprocess imgs
inceptionv3_train_img = preprocess_input_incep_xcep(train_img.astype(np.float32))

# train dense model on folds
performance2 = train_skmodel_ft(rand_forest, base_model, inceptionv3_train_img, train_labels, model_name, init_weights_model, kfolds)

In [ ]:
performance2

with proper normalization data aug and some conv retraining should yield very good results.